<a href="https://colab.research.google.com/github/OmriMan/EnsembleLearning_EnsembleDiversity_Assignment4/blob/main/Ensemble_Ass4_Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/chazzer/ml-grandmaster-decision-tree-classifier

Packages

In [14]:
import numpy as np  
import pandas as pd  

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from matplotlib import pyplot as plt  
from sklearn.tree import plot_tree 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

import random
from tabulate import tabulate

#Dataset -Titanic

Source: https://www.kaggle.com/competitions/titanic/data

survival: Survival, 0 = No, 1 = Yes

pclass: Ticket class, 1 = 1st, 2 = 2nd, 3 = 3rd

sex: Sex. Female = 1,Male = 0

Age: Age in years.Missing values ​​were replaced by the mean ages in the data set

sibsp: numer of siblings / spouses aboard the Titanic

parch: number of parents / children aboard the Titanic

ticket: Ticket number.We did not use this feature

fare: Passenger fare.Missing values ​​were replaced by the mean fare values in the data set

cabin: Cabin number.We did not use this feature

embarked: Port of Embarkation C=Cherbourg,Q=Queenstown,S=Southampton.We replaced the values ​​as follows: S=0,C=1,Q=2

In [15]:
def preprocess(df):
    # Putting on index to each dataset before split it
    df = df.set_index("PassengerId")
    # Sex
    change = {'female':1,'male':0}
    df.Sex = df.Sex.map(change)

    # Embarked
    change = {'S':0,'C':1,'Q':2}
    df.Embarked = df.Embarked.map(change)
    df['Embarked'] = df['Embarked'].replace(to_replace = np.nan, value =1)

    # df['Age'] = df['Age'].groupby(df['Age'].isnull()).mean()
    df['Age'] = df['Age'].replace(to_replace = np.nan, value =df['Age'].mean())
    # df['Age'] = df['Age'].replace(to_replace = np.nan, value =50)
    df['Fare'] = df['Fare'].replace(to_replace = np.nan, value =df['Fare'].mean())
    df = df.drop(columns=['Name','Ticket','Cabin'])
    return df

Load the data set

In [16]:
df = pd.read_csv('/content/sample_data/titanic.csv')
df.isna().sum()

PassengerId       0
Survived          0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [17]:
df = preprocess(df)
features = ['Pclass',	'Sex',	'Age',	'SibSp',	'Parch',	'Fare',	'Embarked']
X = df.loc[:, features]
y = df.loc[:, ['Survived']]

Split the data into training and testin sets

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size = .75)
X_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
PassengerId,,,,,,,
1140,2,1,29.000000,1,0,26.0000,0.0
534,3,1,29.881138,0,2,22.3583,1.0
460,3,0,29.881138,0,0,7.7500,2.0
1151,3,0,21.000000,0,0,7.7750,0.0
394,1,1,23.000000,1,0,113.2750,1.0
...,...,...,...,...,...,...,...
365,3,0,29.881138,1,0,15.5000,2.0
495,3,0,21.000000,0,0,8.0500,0.0
709,1,1,22.000000,0,0,151.5500,0.0


#Target class distribution


In [19]:
target = df['Survived']
target.value_counts()

0    815
1    494
Name: Survived, dtype: int64

#Define and train the models with default hyper parameters
we are using 100 trees for each ensemble. Leave the other parameters at their default settings.


Random Forest

In [20]:
model_rf = RandomForestClassifier(n_estimators=100)

model_rf.fit(X_train,np.ravel(y_train))

RandomForestClassifier()

In [21]:
inputs = pd.DataFrame(X_test)
prediction_RandomForestClassifier = model_rf.predict(
    inputs
)
accuracy_RandomForestClassifier = accuracy_score(y_test,prediction_RandomForestClassifier)
print(f"Accuracy score of RandomForestClassifier = {accuracy_RandomForestClassifier}")

precision_RandomForestClassifier = precision_score(y_test,prediction_RandomForestClassifier,average='weighted')
print(f"Precision score of RandomForestClassifier = {precision_RandomForestClassifier}")

Accuracy score of RandomForestClassifier = 0.8597560975609756
Precision score of RandomForestClassifier = 0.8599780761852562


Extremely Randomized Trees

In [22]:
model_ExtraTreesClassifier = ExtraTreesClassifier(n_estimators=100)
model_ExtraTreesClassifier.fit(X_train,np.ravel(y_train))

ExtraTreesClassifier()

In [23]:
inputs = pd.DataFrame(X_test)
prediction_ExtraTreesClassifier = model_ExtraTreesClassifier.predict(
    inputs
)
accuracy_ExtraTreesClassifier = accuracy_score(y_test,prediction_ExtraTreesClassifier)
print(f"Accuracy score of ExtraTreesClassifier = {accuracy_ExtraTreesClassifier}")

precision_ExtraTreesClassifier = precision_score(y_test,prediction_ExtraTreesClassifier,average='weighted')
print(f"Precision score of ExtraTreesClassifier = {precision_ExtraTreesClassifier}")

Accuracy score of ExtraTreesClassifier = 0.8323170731707317
Precision score of ExtraTreesClassifier = 0.8336247004088538


#Summary - Titanic dataset

In [24]:
print(tabulate([["Accuracy",accuracy_RandomForestClassifier,accuracy_ExtraTreesClassifier],["Precision",precision_RandomForestClassifier,precision_ExtraTreesClassifier]], headers=["Metric\Model","RandomForestClassifier","ExtraTreesClassifier"], tablefmt="grid"))

+----------------+--------------------------+------------------------+
| Metric\Model   |   RandomForestClassifier |   ExtraTreesClassifier |
+================+==========================+========================+
| Accuracy       |                 0.859756 |               0.832317 |
+----------------+--------------------------+------------------------+
| Precision      |                 0.859978 |               0.833625 |
+----------------+--------------------------+------------------------+
